In [1]:
import docker
client = docker.from_env()

In [8]:
images = client.images

In [9]:
images.list()

[<Image: 'drp_scheduler:latest'>,
 <Image: 'drp_db:latest'>,
 <Image: 'runapsim.latest:latest'>,
 <Image: 'mono:latest'>,
 <Image: 'k8s.gcr.io/kube-proxy:v1.14.0'>,
 <Image: 'k8s.gcr.io/kube-scheduler:v1.14.0'>,
 <Image: 'k8s.gcr.io/kube-apiserver:v1.14.0'>,
 <Image: 'k8s.gcr.io/kube-controller-manager:v1.14.0'>,
 <Image: 'mdillon/postgis:10'>,
 <Image: 'k8s.gcr.io/kube-addon-manager:v9.0'>,
 <Image: 'k8s.gcr.io/coredns:1.3.1'>,
 <Image: 'k8s.gcr.io/kubernetes-dashboard-amd64:v1.10.1'>,
 <Image: 'k8s.gcr.io/etcd:3.3.10'>,
 <Image: 'k8s.gcr.io/k8s-dns-sidecar-amd64:1.14.13'>,
 <Image: 'k8s.gcr.io/k8s-dns-kube-dns-amd64:1.14.13'>,
 <Image: 'k8s.gcr.io/k8s-dns-dnsmasq-nanny-amd64:1.14.13'>,
 <Image: 'continuumio/miniconda3:4.5.4'>,
 <Image: 'k8s.gcr.io/pause:3.1'>,
 <Image: 'gcr.io/k8s-minikube/storage-provisioner:v1.8.1'>,
 <Image: 'gcr.io/google_containers/echoserver:1.4'>]

In [10]:
containers = client.containers

In [12]:
containers.list()

[]

In [192]:
def parse_env_file(path_to_file):
    '''
    Parse a Kimetrica .env file into a dictionary
    '''
    envre = re.compile(r'''^([^\s=]+)=(?:[\s"']*)(.+?)(?:[\s"']*)$''')
    env_var = re.compile(r'\$\{[0-9a-zA-Z_]*\}')
    alpha_num = re.compile(r'[0-9a-zA-Z_]+')
    result = {}
    with open(path_to_file) as ins:
        for line in ins:
            match = envre.match(line)
            if match is not None:
                key = match.group(1)
                val = match.group(2)
                to_replace = env_var.findall(val)
                for v in to_replace:
                    found = result[alpha_num.search(v).group(0)]
                    val = val.replace(v, found)
                result[key] = val
    return result

In [208]:
scheduler = 'drp_scheduler:latest'
db = 'drp_db:latest'
entrypoint="luigi --module models.malnutrition_model.tasks models.malnutrition_model.tasks.RasterToCSV --local-scheduler"
volumes = {'/home/ubuntu/darpa/': {'bind': '/usr/src/app/', 'mode': 'rw'}}
environment = parse_env_file('darpa/kiluigi/.env')
db_ports = {'5432/tcp': 5432}
network_name = "kiluigi"

Create a network for Kiluigi activities. First, remove any lingering networks called `kiluigi`:

In [194]:
for net in client.networks.list():
    if net.name == network_name:
        client.networks.get(net.id).remove()

In [195]:
network = client.networks.create(network_name, driver="bridge")

In [196]:
network.name

'kiluigi'

In [197]:
db_environment = {"APP": environment["APP"],
                  "ENV": environment["ENV"],
                  "PGPASSWORD": environment["PGPASSWORD"],
                  "POSTGRES_PASSWORD": environment["PGPASSWORD"]}

In [198]:
db_container = containers.run(db, environment=db_environment, ports=db_ports, network=network_name, detach=True)

In [199]:
db_container.short_id

'8ca8289c5b'

In [211]:
environment['DATABASE_URL'] = environment['DATABASE_URL'].replace('localhost',db_container.short_id)
environment['PYTHONPATH'] = '/usr/src/app:/usr/src/app/kiluigi'

In [212]:
containers.run(scheduler, 
               environment=environment, 
               volumes=volumes, 
               network=network_name, 
               links={db_container.short_id: None},
               entrypoint=entrypoint)

b'[2019-04-22 19:49:52,104] INFO [luigi:80] logging configured via *.conf file\n[2019-04-22 19:49:52,114] INFO [luigi-interface:579] Informed scheduler that task   models.malnutrition_model.tasks.RasterToCSV_0_5_travel_time___landcover____r_333a86b912   has status   DONE\n[2019-04-22 19:49:52,114] INFO [luigi-interface:171] Done scheduling tasks\n[2019-04-22 19:49:52,114] INFO [luigi-interface:1152] Running Worker with 1 processes\n[2019-04-22 19:49:52,115] INFO [luigi-interface:477] Worker Worker(salt=237989178, workers=1, host=ec69c0c24ed4, username=root, pid=1) was stopped. Shutting down Keep-Alive thread\n[2019-04-22 19:49:52,116] INFO [luigi-interface:173] \n===== Luigi Execution Summary =====\n\nScheduled 1 tasks of which:\n* 1 complete ones were encountered:\n    - 1 models.malnutrition_model.tasks.RasterToCSV(...)\n\nDid not run any tasks\nThis progress looks :) because there were no failed tasks or missing dependencies\n\n===== Luigi Execution Summary =====\n\n'